
## Visual analysis of the 'others'


In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)

import plotly.plotly as py
import plotly.graph_objs as go 
import plotly.colors as pc
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(True)

In [2]:
ps = pd.read_csv('playerscores2017.csv')  # 2017 player box scores
bs = pd.read_csv('boxscores2017.csv')  # 2017 team box scores
len(ps.playDispNm.unique())  # number of unique players

540

In [3]:
# calculate game score
ps['gameScore'] = ps.playPTS + 0.4* ps.playFGM - 0.7 * ps.playFGA - 0.4 * (ps.playFTA - ps.playFTM) + \
                  0.7 * ps.playORB + 0.3 * ps.playDRB + ps.playSTL + 0.7 * ps.playAST + \
                  0.7 * ps.playBLK - 0.4 * ps.playPF - ps.playTO


In [4]:
ptgCnt = ps[['gmDate', 'teamAbbr', 'playDispNm']]\
                    .groupby(['teamAbbr', 'playDispNm'], as_index=False).count()
noTeams = ptgCnt.groupby(['playDispNm'], as_index=False).count()  
traded = noTeams.loc[noTeams.gmDate > 1]  # traded players, ignore for now

gsAvgs = ps[['gameScore', 'teamAbbr', 'playDispNm']]\
                    .groupby(['teamAbbr', 'playDispNm'], as_index=False).mean()
gsAvgs['rank'] = gsAvgs.groupby(['teamAbbr'])['gameScore'].rank(ascending=False)
pLocAvgs = ps[['gameScore', 'teamAbbr', 'playDispNm', 'teamLoc']]\
                    .groupby(['teamAbbr', 'playDispNm', 'teamLoc'], as_index=False).mean()
rankGS = pd.merge(pLocAvgs, gsAvgs, on=['teamAbbr', 'playDispNm'], suffixes=('_loc', '_avg'))

rankGS['locLift'] = (rankGS.gameScore_loc - rankGS.gameScore_avg)*100.0/rankGS.gameScore_avg

In [5]:
rankGS = rankGS.loc[(rankGS.gameScore_avg > 0) & (rankGS.gameScore_loc > 0)]

In [6]:
rankGS.head()

,teamAbbr,playDispNm,teamLoc,gameScore_loc,gameScore_avg,rank,locLift
0,ATL,Andrew White,Away,2.433333,2.240000,19.0,8.630952
1,ATL,Andrew White,Home,1.950000,2.240000,19.0,-12.946429
3,ATL,Antonius Cleveland,Home,2.050000,0.800000,21.0,156.250000
4,ATL,Damion Lee,Away,7.922222,7.886667,8.0,0.450831
5,ATL,Damion Lee,Home,7.833333,7.886667,8.0,-0.676247


In [7]:
colors[1]

/Users/patrick.yang/anaconda3/lib/python3.6/site-packages/IPython/core/magics/basic.py:356: UserWarning:

Error changing shell color schemes.
The 'colors' trait of a ZMQInteractiveShell instance must be any of ['Neutral', 'NoColor', 'LightBG', 'Linux'], but a value of '[1]' <class 'str'> was specified.

/Users/patrick.yang/anaconda3/lib/python3.6/site-packages/IPython/core/magics/basic.py:363: UserWarning:

Error changing exception color schemes.
Unrecognized color scheme: [1]
Valid schemes: ['NoColor', 'Linux', 'LightBG', 'Neutral', '']

/Users/patrick.yang/anaconda3/lib/python3.6/site-packages/IPython/core/magics/basic.py:370: UserWarning:

Error changing object inspector color schemes.
Unrecognized color scheme: [1]
Valid schemes: ['NoColor', 'Linux', 'LightBG', 'Neutral', '']



In [8]:
teams = ps.teamAbbr.unique()
traces = []
colors = pc.DEFAULT_PLOTLY_COLORS
homeGS = rankGS.loc[rankGS.teamLoc == 'Home'].sort_values('rank')
maxRank = 14

for i, team in list(enumerate(teams)):
    traces += [go.Scatter(
                x=homeGS.loc[homeGS.teamAbbr == team, 'rank'][:maxRank],
                y=homeGS.loc[homeGS.teamAbbr == team, 'locLift'][:maxRank],
                name=team,
                text=homeGS.loc[homeGS.teamAbbr == team, 'playDispNm'][:maxRank],
                visible=True,
                opacity = 0.15,
                marker=dict(color = colors[i%len(colors)]),
                hoverinfo='none'
                          )
              ] 
    
leagueAvg = homeGS.groupby(['rank'], as_index=False)['locLift'].mean()

traces += [go.Scatter(
            x=leagueAvg['rank'][:maxRank],
            y=leagueAvg['locLift'][:maxRank],
            name='League Avg Lift',
            visible=True,
            opacity = 1,
            marker=dict(color = 'rgb(0,0,0)')
                      )
          ] 

layout = dict(
    title='Home GmSc Lift as a league Average', 
    showlegend=False,
    xaxis=dict(
        title='Average GmSc Rank within Team'
               ),
    yaxis=dict(
        title='% Lift in GmSc Average at Home'
               ),    
              )

fig = dict(data=traces, layout=layout)


iplot(fig, filename='GmSc League')

In [9]:
teams = ps.teamAbbr.unique()
traces = []
colors = pc.DEFAULT_PLOTLY_COLORS
homeGS = rankGS.loc[rankGS.teamLoc == 'Home'].sort_values('rank')

for i, team in list(enumerate(teams)):
    traces += [go.Scatter(
                x=homeGS.loc[homeGS.teamAbbr == team, 'rank'][:maxRank],
                y=homeGS.loc[homeGS.teamAbbr == team, 'locLift'][:maxRank],
                name=team,
                text=homeGS.loc[homeGS.teamAbbr == team, 'playDispNm'][:maxRank],
                visible=True,
                opacity = 1,
                marker=dict(color = colors[i%len(colors)])
                          )
              ] 
    

buttons = []

for i, team in list(enumerate(teams)):
    buttons += [dict(label = team,
                 method = 'update',
                 args = [{'visible': [(n%30) == i for n in range(60)]},
                         {'title': "%s's Home GmSc lift by rank in team" % team},
                         {'opacity': [0.9 for n in range(60)]}
                         ])]
    
buttons += [dict(label = 'Reset',
                 method = 'update',
                 args = [{'visible': [True for n in range(60)]},
                         {'title': "Home GmSc lift by rank in team"},
                         ]
                )]

updatemenus = list([
    dict(active=-1,
         buttons=buttons,
    )
])


layout = dict(
    title='Home GmSc Lift as a league Average', 
    showlegend=False,
    xaxis=dict(
        title='Average GmSc Rank within Team'
               ),
    yaxis=dict(
        title='% Lift in GmSc Average at Home'
               ),    
    updatemenus=updatemenus
              )
fig = dict(data=traces, layout=layout)
iplot(fig, filename='GmSc by team')